Once the content has been downloaded and split into paragraphs, perform preprocessing

TODO

Parentheticals structured like this need to be addressed
35 suspected measles cases in Idleb (16) Aleppo (5) Daraa (4) Rural Damascus (3) Homs (2) Hama (2) Deir-ez-Zor (2) AlHasakeh (1)



In [1]:
import pandas as pd
import uuid
import json
import os
import uuid
import re
import importlib

from datetime import datetime


import spacy
from textacy import preprocessing

import sys
sys.path.append('utilities')
import basic_utilities as utils
import preprocessing_utilities as preputils
import nlp_utilities as nlputils


import spacy
from textacy import extract
from spacy import displacy

pd.set_option('display.max_colwidth', None)

nlp = spacy.load("en_core_web_md")

In [2]:
in_file = "stored_data\\rw_disaster_situation_reports_8a4af19ed04345f2b41d8de4adb98d4e.xlsx"


In [3]:
df = pd.read_excel(in_file)

## Preprocessing

In [4]:
importlib.reload(preputils)
df['preprocessed_text'] = df['text'].apply(preputils.simple_string_preprocessing)
df['preprocessed_text'] = df['preprocessed_text'].apply(preputils.numeric_string_preprocessing)

#if the field isn't text, drop the record
def clean_main_text_field(text):
    if isinstance(text, str) == False:
        return None
    else:
        return text
        
df['preprocessed_text'] = df['preprocessed_text'].apply(clean_main_text_field)
df = df[df['preprocessed_text'].isna() == False]


## Load Other Controller Stuff

In [5]:
#load indicators
indicator_file = 'stored_data\\word_indicators.xlsx'
df_ind = pd.read_excel(indicator_file)
df_ind.set_index('word', inplace=True)
indicators = {}
for c in df_ind.columns:
    indicators[c] = df_ind[df_ind[c].isna() == False].index.tolist()



## NLP Processing

In [6]:
# NLP the text, break sentences out into their own row

def spacify_text(text):
    if isinstance(text, str):
        return nlp(text)
    else:
        return None


def expand_sentences(df):
    # Function to increment by one for each idx_parad
    def generate_sent_id(group, new_column_name='idx_sent'):
        group[new_column_name] = range(0, len(group))
        return group
    
    df['spacy_doc'] = df['preprocessed_text'].apply(lambda text: spacify_text(text))

    # inefficient to write out to text, just to spacify again...
    df['spacy_doc'] = df['spacy_doc'].apply(lambda x: [sent.text for sent in x.sents])
    df = df.explode('spacy_doc')
    df.reset_index(drop=True, inplace=True)

    df = df.groupby(['doc_id','idx_para']).apply(generate_sent_id).reset_index(drop=True)

    # inefficient to write out to text, just to spacify again...
    df['spacy_doc'] = df['spacy_doc'].apply(lambda text: spacify_text(text))
    df['record_id'] = df['doc_id'].astype(str) + "_" +  df['idx_para'].apply(lambda x: f"{x:04d}") + "_" + df['idx_sent'].apply(lambda x: f"{x:04d}")

    #reorg column headers
    df = df[['record_type','source_url','glide_id','doc_id','idx_para','idx_sent','record_id','source_level_country','source_title','source_desc','source_original_text','reference_url','authoring_org','reported_date','preprocessed_text','spacy_doc']]
    
    return df

df = expand_sentences(df)


In [7]:
df = df[df['spacy_doc'].isna() == False].copy()


In [9]:
def get_future_tense_verb(doc):
    def is_future_tense(token):
        #Check if a token is indicative of future tense.
        return (
            token.tag_ == "MD" and token.text.lower() == "will"
            or (token.dep_ == "aux" and token.head.lemma_ == "will")
        )

In [10]:
df['lower_lemmas'] = df['spacy_doc'].apply(lambda x: [w.lemma_.lower() for w in x])
 

df['locations'] = df['spacy_doc'].apply(lambda doc: [e.text for e in doc.ents if e.label_ == 'GPE'])
df['dates'] = df['spacy_doc'].apply(lambda doc: [e.text for e in doc.ents if e.label_ == 'DATE'])
df['svot'] = df['spacy_doc'].apply(lambda doc: list(extract.subject_verb_object_triples(doc)))
df['future_verbs'] = df['spacy_doc'].apply(get_future_tense_verb)

In [70]:
def find_and_add_indicator(df, indicators):
        
    ind_counter = []
    for ind in indicators:
  
        df[ind] = df['lower_lemmas'].apply(lambda x: 1 if len([w for w in x if w in indicators[ind]])>0 else 0)
        ind_counter.append(ind)
        #print(ind_counter)
    df['i_count'] = df[ind_counter].sum(axis=1)

    return df

df = find_and_add_indicator(df, indicators)

In [44]:
def get_cardinalities(doc):
    if doc is None:
        return None

    cardinalities = []
    for e in doc.ents:
        if e.label_ == "CARDINAL": # it's a cardinal, find out what it's related to
            for t in e:
                if t.pos_ == 'NUM':
                    numeric_token = t
                    object = t.head
                    cardinalities.append([numeric_token, object, object.head])
    if len(cardinalities) > 0:
        return cardinalities
    else:
        return None

#get_cardinalities(doc)      

In [89]:
df['cardinalities'] = df['spacy_doc'].apply(get_cardinalities)

In [90]:
def df_exploder(df, explode_field):
    tmp_df = df.explode(explode_field)
    tmp_df.reset_index(drop=True, inplace=True)
    return tmp_df


df_cardinalities = df[['record_id','spacy_doc','cardinalities']].copy()
df_cardinalities = df_exploder(df_cardinalities, 'cardinalities')
df_cardinalities = df_cardinalities[df_cardinalities['cardinalities'].isna() == False].copy()



In [94]:
df_cardinalities[['one','two','three']] = df_cardinalities['cardinalities'].apply(pd.Series)
df_cardinalities['lower_lemma'] = df_cardinalities['three'].apply(lambda x: x.lemma_.lower())
#had to split out the function to make the ll a list to make it work...
df_cardinalities['lower_lemmas'] = df_cardinalities['lower_lemma'].apply(lambda x: [x]) 
df_cardinalities = find_and_add_indicator(df_cardinalities, indicators)

## END

In [132]:
df.to_excel("c://temp//processed_disaster_summaries.xlsx")